In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = [
    'post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
                 'rnd_padding', 
                 'aug_padding',
                "zoom_padding"
]
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["aug_padding"]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/stack_conv/10filts_sizeJurtz/'#rnn_conv/256rnn/'
#1conv/64filts_size5/' #'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 7 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [313,76]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 10 #64 #None
kernel_size = [1,3,5,9,15]#[5] #None
pool_size = 10 #None
n_hid = 256 #None

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task2/"
#architecture = "only_denses"
#architecture = "conv_dense"
#architecture = "rnn_conv"
architecture = "stack_conv"

In [3]:
lr=0.0001
optimizer = Adam(lr=lr)

### Loading stuff

In [4]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model - Task 2

In [5]:
#model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
#model = building_1convdense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, kernel_size, final_act, folder)
#model = building_convrnn_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 
#                                 n_filt, kernel_size, pool_size, n_hid, final_act, folder, optimizer)
model = building_stackconv_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, 
                                       kernel_size, pool_size, final_act, folder, optimizer)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 10)          270       
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 313)         3443      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000, 76)          23864     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 76)          0         
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, 
                        val_generator, architecture, max_lenn, dict_size, batch_size, 
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val,
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1000, 10)          270       
_________________________________________________________________
dense_4 (Dense)              (None, 1000, 313)         3443      
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000, 76)          23864     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000, 76)          0         
__________

138/138 [==============================] - 3s 25ms/step - loss: 0.2015 - acc: 0.4075 - val_loss: 0.0742 - val_acc: 0.2126

Epoch 00025: val_acc improved from 0.20690 to 0.21264, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task2/post_padding/0/weights-improvement-025-0.2126.hdf5
Epoch 26/200
138/138 [==============================] - 3s 25ms/step - loss: 0.1891 - acc: 0.4124 - val_loss: 0.0645 - val_acc: 0.2133

Epoch 00026: val_acc improved from 0.21264 to 0.21328, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task2/post_padding/0/weights-improvement-026-0.2133.hdf5
Epoch 27/200
138/138 [==============================] - 3s 25ms/step - loss: 0.1818 - acc: 0.4164 - val_loss: 0.0613 - val_acc: 0.2139

Epoch 00027: val_acc improved from 0.21328 to 0.21392, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task2/post_pad

138/138 [==============================] - 3s 25ms/step - loss: 0.0728 - acc: 0.4576 - val_loss: 0.0119 - val_acc: 0.2209

Epoch 00060: val_acc did not improve from 0.23436
Epoch 61/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0723 - acc: 0.4568 - val_loss: 0.0181 - val_acc: 0.2165

Epoch 00061: val_acc did not improve from 0.23436
Epoch 62/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0736 - acc: 0.4540 - val_loss: 0.0143 - val_acc: 0.2184

Epoch 00062: val_acc did not improve from 0.23436
Epoch 63/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0692 - acc: 0.4593 - val_loss: 0.0177 - val_acc: 0.2126

Epoch 00063: val_acc did not improve from 0.23436
Epoch 64/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0724 - acc: 0.4544 - val_loss: 0.0199 - val_acc: 0.2235

Epoch 00064: val_acc did not improve from 0.23436
Epoch 65/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0696

138/138 [==============================] - 3s 25ms/step - loss: 0.0584 - acc: 0.4601 - val_loss: 0.0070 - val_acc: 0.2107

Epoch 00104: val_acc did not improve from 0.23436
Epoch 105/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0525 - acc: 0.4550 - val_loss: 0.0139 - val_acc: 0.2235

Epoch 00105: val_acc did not improve from 0.23436
Epoch 106/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0504 - acc: 0.4718 - val_loss: 0.0076 - val_acc: 0.2126

Epoch 00106: val_acc did not improve from 0.23436
Epoch 107/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0524 - acc: 0.4605 - val_loss: 0.0109 - val_acc: 0.2114

Epoch 00107: val_acc did not improve from 0.23436
Epoch 108/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0507 - acc: 0.4597 - val_loss: 0.0118 - val_acc: 0.2165

Epoch 00108: val_acc did not improve from 0.23436
Epoch 109/200
138/138 [==============================] - 3s 25ms/step - loss: 0

138/138 [==============================] - 3s 25ms/step - loss: 0.0470 - acc: 0.4576 - val_loss: 0.0090 - val_acc: 0.2120

Epoch 00148: val_acc did not improve from 0.23436
Epoch 149/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0468 - acc: 0.4618 - val_loss: 0.0098 - val_acc: 0.2088

Epoch 00149: val_acc did not improve from 0.23436
Epoch 150/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0421 - acc: 0.4614 - val_loss: 0.0060 - val_acc: 0.2133

Epoch 00150: val_acc did not improve from 0.23436
Epoch 151/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0458 - acc: 0.4652 - val_loss: 0.0094 - val_acc: 0.2101

Epoch 00151: val_acc did not improve from 0.23436
Epoch 152/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0455 - acc: 0.4604 - val_loss: 0.0153 - val_acc: 0.2069

Epoch 00152: val_acc did not improve from 0.23436
Epoch 153/200
138/138 [==============================] - 3s 25ms/step - loss: 0

...

138/138 [==============================] - 4s 26ms/step - loss: 0.0451 - acc: 0.4608 - val_loss: 0.0062 - val_acc: 0.2114

Epoch 00190: val_acc did not improve from 0.22414
Epoch 191/200
138/138 [==============================] - 4s 25ms/step - loss: 0.0439 - acc: 0.4575 - val_loss: 0.0070 - val_acc: 0.2152

Epoch 00191: val_acc did not improve from 0.22414
Epoch 192/200
138/138 [==============================] - 4s 25ms/step - loss: 0.0435 - acc: 0.4663 - val_loss: 0.0093 - val_acc: 0.2120

Epoch 00192: val_acc did not improve from 0.22414
Epoch 193/200
138/138 [==============================] - 4s 26ms/step - loss: 0.0482 - acc: 0.4600 - val_loss: 0.0099 - val_acc: 0.2120

Epoch 00193: val_acc did not improve from 0.22414
Epoch 194/200
138/138 [==============================] - 4s 26ms/step - loss: 0.0450 - acc: 0.4588 - val_loss: 0.0090 - val_acc: 0.2139

Epoch 00194: val_acc did not improve from 0.22414
Epoch 195/200
138/138 [==============================] - 4s 26ms/step - loss: 0


Epoch 00017: val_acc did not improve from 0.22031
Epoch 18/200
138/138 [==============================] - 3s 25ms/step - loss: 0.4429 - acc: 0.3175 - val_loss: 0.1480 - val_acc: 0.2209

Epoch 00018: val_acc improved from 0.22031 to 0.22095, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task2/post_padding/2/weights-improvement-018-0.2209.hdf5
Epoch 19/200
138/138 [==============================] - 4s 25ms/step - loss: 0.4173 - acc: 0.3227 - val_loss: 0.1399 - val_acc: 0.2190

Epoch 00019: val_acc did not improve from 0.22095
Epoch 20/200
138/138 [==============================] - 3s 25ms/step - loss: 0.4023 - acc: 0.3327 - val_loss: 0.1314 - val_acc: 0.2165

Epoch 00020: val_acc did not improve from 0.22095
Epoch 21/200
138/138 [==============================] - 3s 25ms/step - loss: 0.3826 - acc: 0.3404 - val_loss: 0.1241 - val_acc: 0.2197

Epoch 00021: val_acc did not improve from 0.22095
Epoch 22/200
138/138 [=================

138/138 [==============================] - 3s 25ms/step - loss: 0.1210 - acc: 0.4383 - val_loss: 0.0250 - val_acc: 0.2280

Epoch 00058: val_acc did not improve from 0.22925
Epoch 59/200
138/138 [==============================] - 3s 25ms/step - loss: 0.1193 - acc: 0.4420 - val_loss: 0.0238 - val_acc: 0.2261

Epoch 00059: val_acc did not improve from 0.22925
Epoch 60/200
138/138 [==============================] - 3s 25ms/step - loss: 0.1167 - acc: 0.4410 - val_loss: 0.0230 - val_acc: 0.2171

Epoch 00060: val_acc did not improve from 0.22925
Epoch 61/200
138/138 [==============================] - 3s 25ms/step - loss: 0.1188 - acc: 0.4373 - val_loss: 0.0218 - val_acc: 0.2248

Epoch 00061: val_acc did not improve from 0.22925
Epoch 62/200
138/138 [==============================] - 3s 25ms/step - loss: 0.1235 - acc: 0.4388 - val_loss: 0.0208 - val_acc: 0.2190

Epoch 00062: val_acc did not improve from 0.22925
Epoch 63/200
138/138 [==============================] - 3s 25ms/step - loss: 0.1132

138/138 [==============================] - 4s 25ms/step - loss: 0.0763 - acc: 0.4528 - val_loss: 0.0135 - val_acc: 0.2184

Epoch 00102: val_acc did not improve from 0.22925
Epoch 103/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0734 - acc: 0.4491 - val_loss: 0.0102 - val_acc: 0.2209

Epoch 00103: val_acc did not improve from 0.22925
Epoch 104/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0783 - acc: 0.4538 - val_loss: 0.0139 - val_acc: 0.2203

Epoch 00104: val_acc did not improve from 0.22925
Epoch 105/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0748 - acc: 0.4540 - val_loss: 0.0118 - val_acc: 0.2120

Epoch 00105: val_acc did not improve from 0.22925
Epoch 106/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0724 - acc: 0.4494 - val_loss: 0.0127 - val_acc: 0.2216

Epoch 00106: val_acc did not improve from 0.22925
Epoch 107/200
138/138 [==============================] - 3s 25ms/step - loss: 0

138/138 [==============================] - 3s 25ms/step - loss: 0.0610 - acc: 0.4560 - val_loss: 0.0161 - val_acc: 0.2171

Epoch 00144: val_acc did not improve from 0.22925
Epoch 145/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0639 - acc: 0.4571 - val_loss: 0.0089 - val_acc: 0.2248

Epoch 00145: val_acc did not improve from 0.22925
Epoch 146/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0618 - acc: 0.4572 - val_loss: 0.0110 - val_acc: 0.2158

Epoch 00146: val_acc did not improve from 0.22925
Epoch 147/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0590 - acc: 0.4569 - val_loss: 0.0129 - val_acc: 0.2171

Epoch 00147: val_acc did not improve from 0.22925
Epoch 148/200
138/138 [==============================] - 4s 25ms/step - loss: 0.0642 - acc: 0.4591 - val_loss: 0.0127 - val_acc: 0.2152

Epoch 00148: val_acc did not improve from 0.22925
Epoch 149/200
138/138 [==============================] - 3s 25ms/step - loss: 0